In [1]:
#!/usr/bin/env pytho
import time
import numpy as np
from ibllib.atlas.regions import BrainRegions
from one.api import ONE
import ibllib.atlas as atlas
import pickle

# have to do this because the Examples folder is below the unitymouse folder
import sys
sys.path.append("..") 
import unitymouse.render as umr

"""
Display all trajectories in one single brain volume
Author: Mayo, Gaelle, Dan
"""

import numpy as np

one = ONE()
ba = atlas.AllenAtlas(25)
traj_rep = one.alyx.rest('trajectories', 'list', provenance='Planned',
    x=-2243, y=-2000,  project='ibl_neuropixel_brainwide_01', use_cache=False)


In [6]:
data = {}

for traj in traj_rep:

    if traj['probe_insertion'] not in except_pid:
        temp_traj = one.alyx.rest('trajectories', 'list',
            provenance='Ephys aligned histology track',
            probe_insertion=traj['probe_insertion'], use_cache=False)
        if len(temp_traj) == 0:
            temp_traj = one.alyx.rest('trajectories', 'list', provenance='Histology track',
                probe_insertion=traj['probe_insertion'], use_cache=False)

            if len(temp_traj) == 0:
                continue

        if not temp_traj[0]['x']:
            continue

        ins = atlas.Insertion.from_dict(temp_traj[0])

        entry_coords = ba.xyz2ccf(ins.entry)
        tip_coords = ba.xyz2ccf(ins.tip)
        recording_size = np.sqrt(np.sum(np.power(entry_coords-tip_coords,2)))
        tip_angles = [ins.phi, ins.theta, ins.beta]
        data[traj['probe_insertion']] = (tip_coords, tip_angles, recording_size)
    
pickle.dump(data,open( './ibl_repro_ephys_traj.pkl', "wb" ))

In [2]:
umr.setup()

UnityMouse Renderer connected to server
Login sent with ID: Dan


In [3]:
data = pickle.load( open( "./data/gaelle/ibl_repro_ephys_traj.pkl", "rb" ) )


In [11]:
umr.clear()

In [14]:
# TODO removing PID manually for sake of figure, but need to iron this out in Alyx
except_pid = ['8b735d77-b77b-4243-8821-37802bf402fe',
'94af9073-0914-4323-a90a-5eea1ef5f92c']

# temp_traj_rep = [traj_rep[0]]
count = 0

for i, (pid, traj) in enumerate(data.items()):
    tip_coords = traj[0]
    tip_angles = traj[1]
    recording_size = traj[2]
    
    probename = 'p'+str(count)
    count+=1
    umr.create_probes([probename])
    umr.set_probe_positions({probename:tip_coords.tolist()})
    umr.set_probe_colors({probename:'#F28C28'})
    umr.set_probe_angles({probename:tip_angles})
    umr.set_probe_size({probename:[0.07,recording_size/1000,0.02]})

vdata = {'VISa': True, 'CA1': True, 'DG': True, 'LP': True, 'PO': True}
umr.set_volume_visibility(vdata)
vdata = {'VISa': 'left', 'CA1': 'left', 'DG': 'left', 'LP': 'left', 'PO': 'left'}
umr.set_volume_style(vdata)
vdata = {'VISa': 'transparent-lit', 'CA1': 'transparent-lit', 'DG': 'transparent-lit', 'LP': 'transparent-lit', 'PO': 'transparent-lit'}
umr.set_volume_shader(vdata)
vdata = {'VISa': 0.4, 'CA1': 0.4, 'DG': 0.4, 'LP': 0.4, 'PO': 0.4}
umr.set_volume_alpha(vdata)

# for some reason 'root' doesn't work?
umr.set_volume_visibility({8:True})
umr.set_volume_shader({8:'transparent-unlit'})
umr.set_volume_color({8:'#000000'})
umr.set_volume_alpha({8:0.025})

In [15]:
umr.close()

UnityMouse Renderer connected to server
Login sent with ID: Dan
